In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install boto3

In [ ]:
!pip install awscli
!aws configure

In [ ]:
import boto3
import json

# Set up Bedrock runtime client
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Define the Claude Haiku model ID
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# Define input messages
messages = [
    {"role": "user", "content": "Hello, can you summarize what LLMs are in plain English?"}
]

# Define request body
body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 300,
    "messages": messages,
    "temperature": 0.7,
    "top_p": 0.9
}

# Invoke Claude Haiku
response = bedrock_client.invoke_model(
    modelId=model_id,
    accept="application/json",
    contentType="application/json",
    body=json.dumps(body)
)

# Parse and print response
response_body = json.loads(response["body"].read())
print(json.dumps(response_body, indent=4))


In [ ]:
response_body

In [ ]:
!aws s3 cp s3://russian-dataset-cisteam/russian-images-archive-s3/92.png .

In [ ]:
test_image = "92.png"
def encode_image(img_file):
    with open(img_file, "rb") as image_file:
        img_str = base64.b64encode(image_file.read())
        base64_string = img_str.decode("latin1")
    return base64_string

from PIL import Image
import base64

raw_image = Image.open(test_image).convert('RGB')
display(raw_image.resize((468, 388)))

In [ ]:
# Define input messages
messages = [
    {"role": "user",
     "content": [
            { "type": "image", "source": { "type": "base64", "media_type": "image/png", "data": encode_image(test_image) } },
            { "type": "text", "text": "What's the content of the image in details?" }
      ]}
]

import boto3
import json

# Create a Boto3 client for Bedrock Runtime
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Define the prompt and other parameters
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'
accept = 'application/json'
contentType = 'application/json'

# Define request body
body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 200,
    "messages": messages,
    "temperature": 0.7,
    "top_p": 0.9
}

# Invoke Claude Haiku
response = bedrock_client.invoke_model(
    modelId=model_id,
    accept="application/json",
    contentType="application/json",
    body=json.dumps(body)
)

# Process the response
response_body = json.loads(response.get('body').read())
print(response_body['content'][0]['text'])


In [ ]:
response_body

In [ ]:
import pandas as pd
import requests
from PIL import Image
import base64
import os

# Load the dataset
df = pd.read_csv("final_russian_processed_testing_dataset.csv")

# Create a folder to save the downloaded images
os.makedirs("images", exist_ok=True)

# Function to download and save an image
def download_image(image_url, save_path):
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(save_path, "wb") as f:
            f.write(response.content)
        return save_path
    else:
        print(f"Failed to download {image_url}")
        return None

# Download a single image and verify
sample_row = df.iloc[102]  # Select the first row for testing
image_url = sample_row["image_url"]
image_path = f"images/{sample_row['id']}.jpg"  # Save as ID.jpg

# Download and check
downloaded_image_path = download_image(image_url, image_path)
if downloaded_image_path:
    print(f"Downloaded image saved at: {downloaded_image_path}")

    # Open and display the image
    image = Image.open(downloaded_image_path).convert("RGB")
    display(image.resize((468, 388)))


In [ ]:
import boto3
import json

# AWS Bedrock client
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Claude Haiku model ID
model_id = 'anthropic.claude-3-haiku-20240307-v1:0'

# Define the system prompt
system_prompt = "You are a helpful assistant that answers with only '1' for sarcastic or '0' for not sarcastic when analyzing the text and image."
# justification_system_prompt = "You are a helpful assistant that answers with only '1' for sarcastic or '0' for not sarcastic when analyzing the text and image. Then, justify why it is sarcastic or not"

# Encode the image in Base64
def encode_image(img_file):
    with open(img_file, "rb") as image_file:
        img_str = base64.b64encode(image_file.read())
        return img_str.decode("latin1")

# Prepare input
text = sample_row["text"]
encoded_image = encode_image(downloaded_image_path)

# Create the message
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": system_prompt},  # Provide instructions
            {"type": "text", "text": text},  # Actual text from the dataset
            {"type": "image", "source": {"type": "base64", "media_type": "image/jpeg", "data": encoded_image}}  # Image content
        ]
    }
]

# Define request body
body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens":2,
    "messages": messages,
    "temperature": 0.1,
    "top_p": 0.9
}

# Invoke Claude Haiku
response = bedrock_client.invoke_model(
    modelId=model_id,
    accept="application/json",
    contentType="application/json",
    body=json.dumps(body)
)

# Process the response
response_body = json.loads(response["body"].read())

if "content" in response_body and response_body["content"]:
    print("Claude's Prediction:", response_body["content"][0]["text"])
else:
    print("No content returned by Claude. Response:", json.dumps(response_body, indent=4))


In [ ]:
import boto3
import json
import pandas as pd
import requests
from PIL import Image
import base64
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Load dataset
df_o = pd.read_csv("final_russian_processed_testing_dataset.csv")

# Split dataset into 70% training and 30% testing
train_df, df = train_test_split(df_o, test_size=0.3, random_state=42)

# AWS Bedrock client
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Claude Haiku model ID
model_id = 'anthropic.claude-3-haiku-20240307-v1:0'

# Define the system prompt
system_prompt = "You are a helpful assistant that answers with only '1' for sarcastic or '0' for not sarcastic when analyzing the text and image."

# Function to download and save an image
def download_image(image_url, image_id, folder="images"):
    response = requests.get(image_url)
    if response.status_code == 200:
        file_ext = "jpg" if "jpg" in image_url else "png"
        image_path = f"{folder}/{image_id}.{file_ext}"
        with open(image_path, "wb") as f:
            f.write(response.content)
        return image_path
    return None

# Function to encode image in Base64
def encode_image(img_file):
    with open(img_file, "rb") as image_file:
        img_str = base64.b64encode(image_file.read())
        return img_str.decode("latin1")

# Initialize results
true_labels = []
predictions = []

# Loop through dataset
for index, row in df.iterrows():
    try:
        # Download image
        image_path = download_image(row["image_url"], row["id"])
        if not image_path:
            print(f"Skipping row {index}: Image not available.")
            continue

        # Encode the image
        encoded_image = encode_image(image_path)

        # Prepare input
        text = row["text"]
        true_label = row["label"]

        # Create the message
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": system_prompt},
                    {"type": "text", "text": text},
                    {"type": "image", "source": {"type": "base64", "media_type": "image/jpeg", "data": encoded_image}}
                ]
            }
        ]

        # Define request body
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 2,
            "messages": messages,
            "temperature": 0.1,
            "top_p": 0.9
        }

        # Invoke Claude Haiku
        response = bedrock_client.invoke_model(
            modelId=model_id,
            accept="application/json",
            contentType="application/json",
            body=json.dumps(body)
        )

        # Process the response
        response_body = json.loads(response["body"].read())
        if "content" in response_body and response_body["content"]:
            predicted_label = int(response_body["content"][0]["text"].strip())
            true_labels.append(true_label)
            predictions.append(predicted_label)
            print(f"Processed row {index}: True Label={true_label}, Predicted Label={predicted_label}")
        else:
            print(f"Skipping row {index}: No content returned by Claude.")

    except Exception as e:
        print(f"Error processing row {index}: {e}")

In [ ]:
# Generate classification report
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=["Not Sarcastic", "Sarcastic"]))

# Calculate and display accuracy
accuracy = sum([1 for t, p in zip(true_labels, predictions) if t == p]) / len(true_labels)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
import json
import os
import pandas as pd
import requests
from PIL import Image
import base64
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Load dataset
df_o = pd.read_csv("processed_data_extended_with_links.csv")

# Split dataset into 70% training and 30% testing
train_df, df = train_test_split(df_o, test_size=0.3, random_state=42)

In [ ]:
# Filter dataset to ensure valid image paths
def preprocess_dataset(df, image_folder="images"):
    processed_data = []
    for _, row in df.iterrows():
        image_url = row["image_url"]
        save_path = f"{image_folder}/{row['id']}.jpg"  # Adjust based on dataset
        downloaded_path = download_image(image_url, save_path)

        if downloaded_path:  # Only include valid rows
            processed_data.append({
                "text": row["text"],
                "image_path": downloaded_path,
                "label": row["label"]
            })
        else:
            print(f"Skipping row with invalid image: id={row['id']}, url={image_url}")

    return processed_data

# Process the dataset
processed_dataset = preprocess_dataset(df)

In [ ]:
print(f"Total valid rows: {len(processed_dataset)}")

Total valid rows: 60


In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Prepare a custom dataset
# class MultimodalDataset(Dataset):
#     def __init__(self, dataframe):
#         self.data = dataframe

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         row = self.data.iloc[idx]
#         text = row['text']
#         label = row['label']
#         image_path = download_image(row['image_url'], row['id'])
#         return {"text": text, "label": label, "image_path": image_path}

class MultimodalDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "text": item["text"],
            "image_path": item["image_path"],
            "label": torch.tensor(item["label"], dtype=torch.long)
        }

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "text": item["text"],
            "image_path": item["image_path"],
            "label": torch.tensor(item["label"], dtype=torch.long)
        }

# Initialize dataset and dataloader
# train_dataset = MultimodalDataset(train_df)
# train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
train_dataset = CustomDataset(processed_dataset)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [ ]:
import torch.nn as nn
from transformers import CLIPModel

class CustomCLIPModel(nn.Module):
    def __init__(self):
        super(CustomCLIPModel, self).__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.fc = nn.Linear(self.clip.config.projection_dim, 2)  # Binary classification layer

    def forward(self, input_ids, attention_mask, pixel_values):
        # Forward pass through CLIP
        outputs = self.clip(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)

        # Extract logits_per_image tensor
        logits_per_image = outputs.logits_per_image  # [batch_size, projection_dim]

        # Pass through classification layer
        logits = self.fc(logits_per_image)  # [batch_size, 2]
        print(logits)
        return logits


In [ ]:
# Replace the model with the custom one
model = CustomCLIPModel()

# Freeze original CLIP layers
for param in model.clip.parameters():
    param.requires_grad = False


In [ ]:
import os
import requests

def download_image(image_url, save_path):
    try:
        # Ensure inputs are valid strings
        if not isinstance(image_url, str) or not isinstance(save_path, str):
            print(f"Invalid input: image_url={image_url}, save_path={save_path}")
            return None

        # Ensure the directory exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        # Download the image
        response = requests.get(image_url)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
            return save_path
        else:
            print(f"Failed to download image: {image_url}, status_code={response.status_code}")
            return None
    except Exception as e:
        print(f"Error downloading image: {e}")
        return None


In [ ]:
from transformers import CLIPModel, CLIPProcessor
import torch
from torch.nn import CrossEntropyLoss, Linear
from torch.optim import AdamW

# Load pretrained CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

# Define a Linear layer to classify the outputs
fc = Linear(model.config.projection_dim, 2)  # Assuming binary classification
fc.to(device)  # Move the linear layer to the same device as the model

for epoch in range(5):  # Number of epochs
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        # Prepare inputs
        inputs = processor(
            text=batch["text"],
            images=[Image.open(p) for p in batch["image_path"]],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=77
        )

        # Move inputs and labels to the appropriate device
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)
        pixel_values = inputs["pixel_values"].to(device)
        labels = batch["label"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)
        image_embeddings = outputs.image_embeds   # For image classification

        # # Check logits type and shape
        # print("Logits type:", type(logits))  # Should be <class 'torch.Tensor'>
        # print("Logits shape before fc:", logits.shape)  # Expected: [batch_size, 2]

        logits = fc(image_embeddings)

        # Compute loss
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")


In [ ]:
# Save the fine-tuned model and the linear layer
torch.save({
    'model_state_dict': model.state_dict(),
    'fc_state_dict': fc.state_dict()
}, "fine_tuned_clip.pth")


In [ ]:
# Load the fine-tuned model and linear layer
checkpoint = torch.load("fine_tuned_clip.pth")
model.load_state_dict(checkpoint['model_state_dict'])
fc.load_state_dict(checkpoint['fc_state_dict'])

model.to(device)
fc.to(device)
model.eval()
fc.eval()


In [ ]:
from sklearn.metrics import classification_report

# Test dataset and dataloader
test_dataset = MultimodalDataset(df)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Evaluate the model
true_labels = []
predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs = processor(
            text=batch["text"],
            images=[Image.open(p) for p in batch["image_path"]],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=77
        )

        # Move inputs and labels to the device
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)
        pixel_values = inputs["pixel_values"].to(device)
        labels = batch["label"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)
        image_embeddings = outputs.image_embeds
        logits = fc(image_embeddings)
        predicted_labels = torch.argmax(logits, dim=1).cpu().tolist()

        # Append true labels and predictions
        true_labels.extend(labels.cpu().tolist())
        predictions.extend(predicted_labels)

# Classification report
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=["Not Sarcastic", "Sarcastic"]))


In [ ]:
df = df.reset_index(drop=True)
print(df.head())
print(df.info())

from torch.utils.data import Dataset

class MultimodalDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]  # Use .iloc for row access
        return {
            "text": row["text"],
            "image_path": row["image_path"],
            "label": torch.tensor(row["label"], dtype=torch.long)
        }


In [ ]:
import os
import requests

# Create directory to store images
os.makedirs("images", exist_ok=True)

def download_image(image_url, image_id):
    try:
        # Determine file extension
        ext = "jpg" if "jpg" in image_id else "png"
        save_path = f"images/{image_id}"

        # Download the image
        response = requests.get(image_url)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
            return save_path
        else:
            print(f"Failed to download image: {image_url}, status_code={response.status_code}")
            return None
    except Exception as e:
        print(f"Error downloading image: {e}")
        return None

# Apply function to download images and create `image_path` column
df["image_path"] = df.apply(lambda row: download_image(row["image_url"], row["image_id"]), axis=1)

# Drop rows where image download failed
df = df.dropna(subset=["image_path"]).reset_index(drop=True)


In [ ]:
for param in model.parameters():
    param.requires_grad = True  # Unfreeze the entire model


In [ ]:
import torch.nn as nn

class MultimodalClassifier(nn.Module):
    def __init__(self, projection_dim, hidden_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(projection_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

classifier = MultimodalClassifier(model.config.projection_dim, 512, 2)
classifier.to(device)


In [ ]:
# Fine-tuning the entire model
for param in model.parameters():
    param.requires_grad = True

optimizer = AdamW([
    {'params': model.parameters(), 'lr': 5e-6},  # Lower LR for CLIP backbone
    {'params': classifier.parameters(), 'lr': 1e-4}  # Higher LR for classifier head
])

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

for epoch in range(10):  # Train for more epochs
    model.train()
    classifier.train()
    total_loss = 0

    for batch in train_dataloader:
        inputs = processor(
            text=batch["text"],
            images=[Image.open(p) for p in batch["image_path"]],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=77
        )
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)
        pixel_values = inputs["pixel_values"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)
        image_embeddings = outputs.image_embeds
        logits = classifier(image_embeddings)

        # Compute loss
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")


In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'fc_state_dict': fc.state_dict()
}, "fine_tuned_clip_multimodal.pth")

# Load the fine-tuned model
checkpoint = torch.load("fine_tuned_clip_multimodal.pth")
model.load_state_dict(checkpoint['model_state_dict'])
fc.load_state_dict(checkpoint['fc_state_dict'])

model.to(device)
fc.to(device)
model.eval()
fc.eval()


In [ ]:
true_labels = []
predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs = processor(
            text=batch["text"],
            images=[Image.open(p) for p in batch["image_path"]],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=77
        )

        # Move inputs to device
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)
        pixel_values = inputs["pixel_values"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)
        image_embeddings = outputs.image_embeds
        logits = fc(image_embeddings)
        predicted_labels = torch.argmax(logits, dim=1).cpu().tolist()

        # Append true labels and predictions
        true_labels.extend(batch["label"].tolist())
        predictions.extend(predicted_labels)

# Classification Report
from sklearn.metrics import classification_report
print(classification_report(true_labels, predictions, target_names=["Not Sarcastic", "Sarcastic"]))


In [ ]:
/// stop for claude again





In [ ]:
from transformers import pipeline, BlipProcessor, BlipForConditionalGeneration, AutoModelForSequenceClassification, AutoTokenizer
from PIL import Image
import torch

# Load BLIP for image captioning
caption_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Load sentiment analysis model for text
text_sentiment_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
text_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
text_sentiment_pipeline = pipeline("sentiment-analysis", model=text_sentiment_model, tokenizer=text_tokenizer)


In [ ]:
MOOD_MAPPING = {
    "POSITIVE": "joyful",
    "NEGATIVE": "tense",
    "NEUTRAL": "calm",
    "DEFAULT": "neutral",  # Fallback if no clear sentiment or match
    "SCENE_BASED": {
        "storm": "chaotic",
        "rain": "gloomy",
        "desk": "calm",
        "beach": "peaceful",
        "crowd": "dynamic",
    }
}

def generate_mood(image_description, text):
    # Infer text sentiment
    text_sentiment, text_score = infer_text_sentiment(text)

    # Start with text sentiment-based mood
    mood = MOOD_MAPPING.get(text_sentiment, MOOD_MAPPING["DEFAULT"])

    # Enhance mood based on specific keywords in the image description
    for keyword, scene_mood in MOOD_MAPPING["SCENE_BASED"].items():
        if keyword in image_description.lower():
            mood = scene_mood
            break  # Prioritize the first matched keyword

    return mood

# Infer sentiment/mood from text
def infer_text_sentiment(text):
    result = text_sentiment_pipeline(text)[0]
    sentiment = result['label']
    score = result['score']
    return sentiment, score



In [ ]:
import boto3
import json
import pandas as pd
import requests
from PIL import Image
import base64
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Load dataset
df_o = pd.read_csv("final_russian_processed_testing_dataset.csv")

# Split dataset into 70% training and 30% testing
train_df, test_df = train_test_split(df_o, test_size=0.1, random_state=42)

# AWS Bedrock client
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Claude Haiku model ID
model_id = 'anthropic.claude-3-haiku-20240307-v1:0'

# Define the few-shot examples
few_shot_examples = [
    {
        "text": "Wow, another Monday. So excited.",
        "image_description": "A messy office with scattered papers and an overflowing trash bin.",
        "label": "1"
    },
    {
        "text": "Finally, a quiet weekend.",
        "image_description": "A crowded park with children playing and loud music from speakers.",
        "label": "1"
    },
    {
        "text": "This meal looks delicious!",
        "image_description": "A burnt steak and wilted vegetables on a plate.",
        "label": "1"
    },
    {
        "text": "Weather is so great today!",
        "image_description": "A flooded street with heavy rain and abandoned vehicles.",
        "label": "1"
    },
    {
        "text": "I just got promoted! Yay!",
        "image_description": "A man is smiling while holding a certificate in front of a cheering crowd.",
        "label": "0"
    },
    {
        "text": "Dinner with family. Perfect night.",
        "image_description": "A cozy dining room with candles and neatly arranged plates.",
        "label": "0"
    },
]


# Prepare the few-shot learning prompt
few_shot_prompt = "You are a helpful assistant trained to analyze text and images together. Your goal is to classify the input as '1' for sarcastic or '0' for not sarcastic. Only output a single number ('1' or '0') without any additional explanation. Use the image description to interpret the text. Here are some examples:\n\n"
for example in few_shot_examples:
    few_shot_prompt += f"Example:\nText: {example['text']}\nImage Description: {example['image_description']}\nLabel: {example['label']}\n\n"

few_shot_prompt += "Now analyze:\n"

# Function to download and save an image
def download_image(image_url, image_id, folder="images"):
    response = requests.get(image_url)
    if response.status_code == 200:
        file_ext = "jpg" if "jpg" in image_url else "png"
        image_path = f"{folder}/{image_id}.{file_ext}"
        with open(image_path, "wb") as f:
            f.write(response.content)
        return image_path
    return None

# Function to encode image in Base64
def encode_image(img_file):
    with open(img_file, "rb") as image_file:
        img_str = base64.b64encode(image_file.read())
        return img_str.decode("latin1")

# Initialize results
true_labels = []
predictions = []


#image captioning

from transformers import BlipProcessor, BlipForConditionalGeneration

# Load BLIP model for image captioning
caption_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


# Function to generate image descriptions
def generate_image_description(image_path):
    raw_image = Image.open(image_path).convert("RGB")
    inputs = caption_processor(raw_image, return_tensors="pt")
    out = caption_model.generate(**inputs, max_length=150, num_beams=10, no_repeat_ngram_size=3,  early_stopping=True)
    return caption_processor.decode(out[0], skip_special_tokens=True)

def enrich_image_description(description, mood):
    return f"The scene shows {description}. The mood appears {mood}."


# Loop through test dataset
for index, row in test_df.iterrows():
    try:
        # Download image
        image_path = download_image(row["image_url"], row["id"])
        if not image_path:
            print(f"Skipping row {index}: Image not available.")
            continue

        # # Encode the image

        # Generate image description
        image_description = generate_image_description(image_path)
        print(f"Processed row {index}: Image Description: {image_description}")

        # Infer mood
        mood = generate_mood(image_description, row["text"])

        enriched_description = enrich_image_description(image_description, mood)
        print(f"Processed row {index}: Enriched Description: {enriched_description}")


        # Prepare input
        text = row["text"]
        true_label = row["label"]

        # Add the specific instance to the few-shot prompt
        full_prompt = few_shot_prompt + f"Text: {text}\nImage Description: {enriched_description}.\nLabel:"

        # Create the message
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": full_prompt},
                ]
            }
        ]

        # Define request body
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 2,
            "messages": messages,
            "temperature": 0.1,
            "top_p": 0.9
        }

        # Invoke Claude Haiku
        response = bedrock_client.invoke_model(
            modelId=model_id,
            accept="application/json",
            contentType="application/json",
            body=json.dumps(body)
        )

        # Process the response
        response_body = json.loads(response["body"].read())
        if "content" in response_body and response_body["content"]:
            predicted_label = int(response_body["content"][0]["text"].strip())
            true_labels.append(true_label)
            predictions.append(predicted_label)
            print(f"Processed row {index}: True Label={true_label}, Predicted Label={predicted_label}")
        else:
            print(f"Skipping row {index}: No content returned by Claude.")

    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Evaluate the model
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=["Not Sarcastic", "Sarcastic"]))


In [ ]:
import boto3
import json
import pandas as pd
import requests
from PIL import Image
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Load dataset
df_o = pd.read_csv("final_russian_processed_testing_dataset.csv")
train_df, test_df = train_test_split(df_o, test_size=0.1, random_state=42)

# AWS Bedrock client
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')
model_id = 'anthropic.claude-3-haiku-20240307-v1:0'

# Initialize results
true_labels = []
predictions = []

# Function to download and save an image
def download_image(image_url, image_id, folder="images"):
    response = requests.get(image_url)
    if response.status_code == 200:
        file_ext = "jpg" if "jpg" in image_url else "png"
        image_path = f"{folder}/{image_id}.{file_ext}"
        with open(image_path, "wb") as f:
            f.write(response.content)
        return image_path
    return None

# First Claude Request: Generate Image Description and Mood
def generate_image_description_and_mood(image_path):
    prompt = (
        "You are an assistant trained to analyze images. "
        "Describe the image in detail, considering all visible elements. "
        "Additionally, infer the mood of the scene using adjectives like 'calm,' 'joyful,' 'chaotic,' etc.\n\n"
        f"Input: An image of {image_path}.\n"
        "Output:"
    )

    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 100,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "top_p": 0.9,
    }

    response = bedrock_client.invoke_model(
        modelId=model_id,
        accept="application/json",
        contentType="application/json",
        body=json.dumps(body)
    )

    response_body = json.loads(response["body"].read())
    return response_body.get("content", "")  # Return raw response

# Second Claude Request: Sarcasm Classification
def classify_sarcasm(text, description, mood):
    prompt = (
        "You are an assistant trained to analyze text and images together. "
        "Classify the input as '1' for sarcastic or '0' for not sarcastic. "
        "Here is the information:\n\n"
        f"Text: {text}\nImage Description: {description}\nMood: {mood}\n\n"
        "Only output a single number ('1' or '0') without any explanation."
    )

    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 2,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.1,
        "top_p": 0.9,
    }

    response = bedrock_client.invoke_model(
        modelId=model_id,
        accept="application/json",
        contentType="application/json",
        body=json.dumps(body)
    )

    response_body = json.loads(response["body"].read())
    return int(response_body.get("content", "0").strip())  # Default to '0' if no response

# Main Loop
for index, row in test_df.iterrows():
    try:
        # Download image
        image_path = download_image(row["image_url"], row["id"])
        if not image_path:
            print(f"Skipping row {index}: Image not available.")
            continue

        # Generate description and mood
        description_and_mood = generate_image_description_and_mood(image_path)
        print(f"Row {index}: Claude Response: {description_and_mood}")

        # Use the raw response for sarcasm classification
        text = row["text"]
        true_label = row["label"]
        sarcasm_prompt = (
            f"You are a helpful assistant. Determine if the following text and image "
            f"response are sarcastic ('1') or not sarcastic ('0').\n\n"
            f"Text: {text}\nImage Analysis: {description_and_mood}\nLabel:"
        )

        # Create the message for sarcasm classification
        messages = [
            {
                "role": "user",
                "content": sarcasm_prompt
            }
        ]

        # Sarcasm classification request
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 2,
            "messages": messages,
            "temperature": 0.1,
            "top_p": 0.9
        }

        response = bedrock_client.invoke_model(
            modelId=model_id,
            accept="application/json",
            contentType="application/json",
            body=json.dumps(body)
        )

        # Extract predicted label
        response_body = json.loads(response["body"].read())
        predicted_label = int(response_body.get("content", "").strip())

        # Append results
        true_labels.append(true_label)
        predictions.append(predicted_label)
        print(f"Processed row {index}: True Label={true_label}, Predicted Label={predicted_label}")

    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Evaluate the model
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=["Not Sarcastic", "Sarcastic"]))
